# **Power Bi Vs Tableau**
## **Which one should you learn first?**

The focus of this analysis is to determine which BI tool one should learn first based on job market trends. In this analysis, I will scrap job data from the following job boards:
* **My job Mag Kenya**:provides local context on market trends
* **Upwork**: provides context on the freelance landscape

After scrapping, I will analyse the deman of each BI tool, by applying a regex function to understand which is the most mentioned.

#### Scrapping data from MyJobMag

In [49]:
#import all necerssary libraries

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium_stealth import stealth

import time
import random

In [50]:
#set up the browser using the options object

options=webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_experimental_option('excludeSwitches',['enable_automation'])
options.add_experimental_option('useAutomationExtension','False')

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

#now setup the webdriver in stealth mode

stealth(driver,
        languages=["en-US", "en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True,
        )

url=('https://www.myjobmag.co.ke/search/jobs?q=business+intelligence+analyst%2C+BI+developer%2C+data+analyst&q=business+intelligence+analyst%2C+BI+developer%2C+data+analyst')
driver.get(url)
time.sleep(random.uniform(4,6))

In [51]:
link_elements = driver.find_elements(By.CSS_SELECTOR, "ul.job-list li.job-list-li li.job-info li.mag-b h2 a")

for element in link_elements:
    print("HTML:", element.get_attribute("outerHTML"))

HTML: <a href="/job/developer-business-intelligence-standard-bank-group-1">Developer, Business Intelligence at Standard Bank Group</a>
HTML: <a href="/job/business-intelligence-bi-developer-apa-life-assurance-company-ltd">Business Intelligence (BI) Developer at APA Life Assurance Company Ltd</a>
HTML: <a href="/job/manager-data-transformation-intelligence-and-impact-dtii-john-snow-inc-jsi">Manager- Data Transformation Intelligence and Impact (DTII) at John Snow Inc (JSI)</a>
HTML: <a href="/job/manager-data-transformation-intelligence-and-impact-insupply-health">Manager – Data Transformation Intelligence and Impact at InSupply Health</a>
HTML: <a href="/job/business-intelligence-analyst-gertrude-s-children-s-hospital">Business Intelligence Analyst at Gertrude's Children's Hospital</a>
HTML: <a href="/job/senior-analyst-business-intelligence-cellulant-corporation">Senior Analyst: Business Intelligence at Cellulant Corporation</a>
HTML: <a href="/job/business-intelligence-analyst-senri-l

In [52]:
job_links=[]

link_elements = driver.find_elements(By.CSS_SELECTOR, "ul.job-list li.job-list-li li.job-info li.mag-b h2 a")


for element in link_elements:
    href=element.get_attribute('href')
    job_links.append(href)
    
print(job_links[0])



https://www.myjobmag.co.ke/job/developer-business-intelligence-standard-bank-group-1


In [54]:
job_data=[]

for i, link in enumerate (job_links):
    try:
        driver.get(link)
        time.sleep(2)

        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASSNAME,"job-details"))
        )
        try:
            title=driver.find_element(By.CSS_SELECTOR,"ul.read-ul li.read-head h1").text.strip()
        except:
            title="N/A"

        try:
            desc = driver.find_element(By.CSS_SELECTOR, "li#printable.job-description div.job-details").text.strip()
        except:
            desc = "N/A"

        job_data.append({
            "title": title,
            "description": desc,
            "link": link
        })
        print(f"scrapped {title}")

    except Exception as e:
        print(f"There is an error loading the link {link}:{e}")
        continue

    time.sleep(1.5)

There is an error loading the link https://www.myjobmag.co.ke/job/developer-business-intelligence-standard-bank-group-1:name 'WebDriverWait' is not defined
There is an error loading the link https://www.myjobmag.co.ke/job/business-intelligence-bi-developer-apa-life-assurance-company-ltd:name 'WebDriverWait' is not defined
There is an error loading the link https://www.myjobmag.co.ke/job/manager-data-transformation-intelligence-and-impact-dtii-john-snow-inc-jsi:name 'WebDriverWait' is not defined
There is an error loading the link https://www.myjobmag.co.ke/job/manager-data-transformation-intelligence-and-impact-insupply-health:name 'WebDriverWait' is not defined


KeyboardInterrupt: 